In [15]:
import numpy as np
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split

timeStamp = []
firstLine = 1


# CONSTANTS
# stores all versions of a signature
signatures = []   
# stores loop interval
target = []

# read all the pictures from a directory
def read_directory(path):
    for file in os.listdir(path):
        # read a signature's descriptors
        firstLine = 1
        signature = []
        coordinates = []
        coordinates2 = []

        with open(path + '/' + file) as ifile:
            for line in ifile:
                if firstLine == 0:
                    splitedLine = line.split(' ')
                    x = splitedLine[0]
                    y = splitedLine[1]
                    # create tuple from coordinates and add to the array
                    signature.append(int(x))
                    signature.append(int(y))
                    coordinates.append((x,y))
                    timeStamp.append(splitedLine[2])
                else:
                    firstLine = 0
                    
            signatures.append(signature)
                     
        # vizsgálandó pontok (pixelek) tömbje, elemei az azonos (x,y) koordinátájú pixelek
        samePixels = []
        
        for r in coordinates:
            coordinates2.append(r)
        
        # kigyűjteni az azonos (x,y) párok indexeit
        for i in range(0,len(coordinates)):
            if coordinates[i] != (-1,-1):
                samePixel = []
                for j in range(i+1,len(coordinates)):
                    #if coordinates[i] == coordinates[j]:
                    if ((int(coordinates[i][0])-600) <= int(coordinates[j][0])) and ((int(coordinates[i][0])+600) >= int(coordinates[j][0])) and ((int(coordinates[i][1])-600) <= int(coordinates[j][1])) and ((int(coordinates[i][1])+600) >= int(coordinates[j][1])):
                        # if it doesn't have any elements
                        if len(samePixel) == 0:
                            samePixel.append(float(i))
                            samePixel.append(float(j))
                        else:
                            samePixel.append(j)
                
                        #coordinates[j] = (-1,-1)
                    
                if len(samePixel) != 0:
                    samePixels.append(samePixel)
        
        # create a single list from array of lists
        samePixels2 = []
        for r in samePixels:
            for s in r:
                samePixels2.append(s)
        
        '''
        samePixels_new = []
        for r in samePixels:
            
            samePixel = []
            coordinate = coordinates2[int(r[0])]
            index = r[0]
            s = 1
            isLoop = 0
            diff = 200
            while s < len(r) and r[s] == (index + 1):
                #if coordinate == coordinates2[int(r[s])]:
                if ((int(coordinate[0])-diff) <= int(coordinates2[int(r[s])][0])) and ((int(coordinate[0])+diff) >= int(coordinates2[int(r[s])][0])) and ((int(coordinate[1])-diff) <= int(coordinates2[int(r[s])][1])) and ((int(coordinate[1])+diff) >= int(coordinates2[int(r[s])][1])):    
                    isLoop = 1
                index = r[s]
                if len(samePixel) == 0:
                    samePixel.append(r[0])
                samePixel.append(index)
                s += 1
            
            if isLoop:
                samePixels_new.append(samePixel)
        
            if s < len(r):
                coordinate = coordinates2[int(r[s])]
                index = r[s]
                s = s + 1
                samePixel = []
                while s <len(r) and r[s] == (index + 1):
                    #if coordinate == coordinates2[int(r[s])]:
                    if ((int(coordinates2[int(r[0])][0])-diff) <= int(coordinates2[int(r[s])][0])) and ((int(coordinates2[int(r[0])][0])+diff) >= int(coordinates2[int(r[s])][0])) and ((int(coordinates2[int(r[0])][1])-diff) <= int(coordinates2[int(r[s])][1])) and ((int(coordinates2[int(r[0])][1])+diff) >= int(coordinates2[int(r[s])][1])):    
                        isLoop = 1
                    index = r[s]
                    if len(samePixel) == 0:
                        samePixel.append(r[s-1])
                    samePixel.append(index)
                    s += 1
                if isLoop:
                    samePixels_new.append(samePixel)
        
        '''
        # check from the end if the first point has it's pair in the end
        final = []
        for r in samePixels:
            if len(r) != 0:
                coordinate = coordinates2[int(r[0])]
                half = len(r) // 2
                isLoop = 0
                diff = 100
                for k in range(len(r)-1,half,-1):
                    if ((int(coordinate[0])-diff) <= int(coordinates2[int(r[k])][0])) and ((int(coordinate[0])+diff) >= int(coordinates2[int(r[k])][0])) and ((int(coordinate[1])-diff) <= int(coordinates2[int(r[k])][1])) and ((int(coordinate[1])+diff) >= int(coordinates2[int(r[k])][1])):    
                        isLoop = 1
                    
                if isLoop:
                    final.append(r)
        
        # remove subset of sequences
        final2 = []
        for r in final:
            element = 0
            for s in final:
                if r != s and r[0] in s: #and r[len(r)-1] in s:
                    element = 1
            if element != 1:
                final2.append(r)
        
        #target.append(samePixels2)
        print(file)
        for s in final2:
            if len(s) >=10:
                print(s)
        #print(samePixels)
        #print(samePixels_new)
 
'''
# tárolja a hurkot alkotó pixelek timestampjét
loops = []

# azonos koordinátájú pixeleknél megnézi, hogy kielégíti-e a hurok feltételt
for i in range(0,len(samePixels)):
    for j in range(0,len(samePixels[i])):
        j_idx = samePixels[i][j]
        
        # ha van elozo pont
        if j_idx - 1 >= 0:
            j_x = coordinates[j_idx][0]
            j_y = coordinates[j_idx][1]
        
            prev_j_x = coordinates[j_idx-1][0]
            #prev_j_y = coordinates[j_idx-1][1]
            
            for k in range(j+1,len(samePixels[i])):
                k_idx = samePixels[i][k]
                
                # ha van kovetkezo pont és nem szomszédos a két azonos koordinátájú pont, mivel akkor lehet csak elidőzött rajta
                if (k_idx + 1 < len(coordinates)): #and (j_idx + 1 != k_idx):
                    post_k_x = coordinates[k_idx+1][0]
                    #post_k_y = coordinates[k_idx+1][1]
                    
                    # hurok-e
                    if (prev_j_x < post_k_x) or (prev_j_x > post_k_x):
                        loops.append(j_idx)
                        loops.append(k_idx)
                   
                
# ha a loops tömb üres, akkor nincs az aláírásban hurok, különben a hurok kezdő és végpontjának indexét tartalmazza
if len(loops) == 0:
    print('Nincs loop')
else:
    print(loops)


#print(samePixels)
'''

read_directory('../Task2/U1')



U4S33.TXT
[0.0, 1.0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
[34.0, 35.0, 36, 37, 38, 39, 49, 50, 51, 52, 53, 54, 55]
[46.0, 47.0, 48, 56, 57, 58, 59, 60, 61, 62, 63, 64]
[71.0, 72.0, 73, 74, 75, 76, 77, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97]
[110.0, 111.0, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126]
[150.0, 151.0, 152, 153, 154, 155, 156, 157, 158, 159, 160, 166, 167, 168, 169, 170, 171, 172]
U4S27.TXT
[3.0, 4.0, 5, 17, 18, 19, 20, 21, 22, 23, 24, 25]
[72.0, 73.0, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 110, 111, 112, 113]
U4S26.TXT
[4.0, 5.0, 6, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
[38.0, 39.0, 40, 41, 42, 43, 44, 45, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65]
[73.0, 74.0, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111]
[160.0, 1

U4S7.TXT
[32.0, 33.0, 34, 35, 39, 47, 48, 49, 50, 51, 52, 55, 56, 57, 58, 59, 60, 71, 72, 73, 74]
[40.0, 41.0, 42, 43, 44, 45, 46, 47, 60, 61, 62, 63, 64, 65, 66, 67]
[80.0, 81.0, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120]
[128.0, 129.0, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145]
U4S6.TXT
[29.0, 30.0, 31, 32, 33, 34, 35, 44, 45, 46, 47, 48, 49, 50, 51]
[67.0, 68.0, 69, 70, 71, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92]
[116.0, 117.0, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133]
U4S4.TXT
[0.0, 1.0, 2, 3, 4, 5, 6, 7, 8, 17, 18, 19, 20, 21, 22, 23]
[27.0, 28.0, 29, 30, 31, 32, 40, 41, 42, 43, 44, 45, 46]
[33.0, 34.0, 35, 36, 37, 38, 39, 40, 47, 48, 49, 50, 51, 52]
[109.0, 110.0, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129]
U4S5.TXT
[29.